In [ ]:
# Import Dependencies
import pandas as pd 
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# import tensorflow as tf

In [ ]:
# Read CSV file from Resources folder into Pandas DataFrame
file = Path('./Resources_sdt/survey.csv')
survey_df = pd.read_csv(file)

# Review DataFrame
survey_df.head()

In [ ]:
# Dropping 'comments' column due to limited value-added data and droppint 'Timestamp' because the entire dataset is for 2014
# Adding option to drop 'state' to look at the dataset globally and due to the NaN values -- can comment out and rerun
survey_df.drop(columns=['comments', 'Timestamp'], inplace=True)
survey_df.drop(columns=['state'], inplace=True)

In [ ]:
# Get DataFrame info
survey_df.info()

In [ ]:
# Determine number of unique values in each column
survey_df.nunique()

## Count Countries
Determine how many, and which, nations are in the dataset, and determine their prevalence.

In [ ]:
# Look at 'Country' value counts
country_count = survey_df['Country'].value_counts()
country_count

## Gender Cleanup
There were 49 unique genders listed. Some of these were because of input errors. Some are because of how the person represented themselves. Need to determine which they are to correct them.

In [ ]:
# Look at 'Gender' value counts
gender_count = survey_df['Gender'].value_counts()
gender_count

In [ ]:
# total 'Gender'
gender_column = survey_df['Gender'].count()
gender_column

In [ ]:
# There may have been misspellings in 'Gender' names and in some cases bogus answers. This is an attempt to cleanup the 'Gender' to form usable data.

# Adding Category for 'Male'
survey_df['Gender'].replace(['Male', 'male', 'M', 'm', 'Male ', 'Cis Male', 'cis male', 'Man', 
                            'Make', 'Mail', 'Cis Man', 'Guy (-ish) ^_^', 'Male-ish', 'maile', 
                            'Mal', 'Male (CIS)', 'msle', 'Malr'], 'Male', inplace=True)

# Adding Category for 'Female'
survey_df['Gender'].replace(['Female', 'female', 'F', 'f', 'Woman', 'Female ', 'Female (cis)', 
                            'cis-female/femme', 'femail', 'Cis Female', 'Femake', 'woman'], 'Female', inplace=True)

# Adding Category for 'Other' -- Note: Other will include non-specific gender and trans
survey_df['Gender'].replace(['Nah', 'All', 'fluid', 'Genderqueer', 'ostensibly male, unsure what that really means', 
                            'non-binary', 'Androgyne', 'Agender', 'Enby', 'p', 'Neuter', 'queer', 'Trans woman', 
                            'Female (trans)', 'male leaning androgynous', 'A little about you', 'Trans-female', 
                            'something kinda male?', 'queer/she/they'], 'Other', inplace=True)

# Check the updated value counts
gender_update = survey_df['Gender'].value_counts()
gender_update

In [ ]:
# check 'Gender' column count
gender_column = survey_df['Gender'].count()
gender_column

## Check Age Ranges in Dataset
Verify ages for authenticity.

In [ ]:
# Look at 'Age' value counts
age_ranges = survey_df['Age'].value_counts()
age_ranges

In [ ]:
# total 'Age'
age_column = survey_df['Age'].count()
age_column

In [ ]:
# Several answers for 'Age' were given with incorrect or bogus values. This is an attempt to retain the data.

# Convert the 'Age' column to numeric
survey_df['Age'] = pd.to_numeric(survey_df['Age'], errors='coerce')

# Then replace any ages less than 0 OR greater than 99 with 'Other'
survey_df.loc[(survey_df['Age'] < 0) | (survey_df['Age'] > 99), 'Age'] = 'Other'

# Check the updated value counts
age_update = survey_df['Age'].value_counts()
age_update

In [ ]:
# check 'Age' column count
age_column = survey_df['Age'].count()
age_column

In [ ]:
# Choose cutoff values and create a list of Age ranges to be replaced
# create the `bin_#_replace`
bin_1_replace = age_update[age_update < 20].index.tolist()
bin_2_replace = age_update[age_update < 25].index.tolist()
bin_3_replace = age_update[age_update < 30].index.tolist()
bin_4_replace = age_update[age_update < 35].index.tolist()
bin_5_replace = age_update[age_update < 40].index.tolist()
bin_6_replace = age_update[age_update < 45].index.tolist()
bin_7_replace = age_update[age_update > 50].index.tolist()
bin_8_replace = age_update[age_update == 'Other'].index.tolist()

# Replace in dataframe
for one in bin_1_replace:
    survey_df['Age'] = survey_df['Age'].replace(one,'1-19')

for two in bin_2_replace:
    survey_df['Age'] = survey_df['Age'].replace(two,'20-24')

for thr in bin_3_replace:
    survey_df['Age'] = survey_df['Age'].replace(thr,'25-29')

for fou in bin_4_replace:
    survey_df['Age'] = survey_df['Age'].replace(fou,'30-34')

for fiv in bin_5_replace:
    survey_df['Age'] = survey_df['Age'].replace(fiv,'40-44')

for six in bin_6_replace:
    survey_df['Age'] = survey_df['Age'].replace(six,'45-49')

for sev in bin_7_replace:
    survey_df['Age'] = survey_df['Age'].replace(sev,'50+')

for egt in bin_8_replace:
    survey_df['Age'] = survey_df['Age'].replace(egt,'Other')

# Check to make sure binning was successful
survey_df['Age'].value_counts()

In [ ]:
# check 'Age' column count
age_column = survey_df['Age'].count()
age_column

## Survey Data
The contains either Yes/No or short answers for the general survey questions.

In [ ]:
# Look at 'self_employed' value counts
self_emp_count = survey_df['self_employed'].value_counts()
self_emp_count

In [ ]:
# total 'self_employed'
self_emp_column = survey_df['self_employed'].count()
self_emp_column

In [ ]:
# Look at 'family_history' value counts
fam_history_count = survey_df['family_history'].value_counts()
fam_history_count

In [ ]:
# total 'family_history'
fam_history_column = survey_df['family_history'].count()
fam_history_column

In [ ]:
# Look at 'treatment' value counts
treatment_count = survey_df['treatment'].value_counts()
treatment_count

In [ ]:
# total look for 'treatment'
treatment_column = survey_df['treatment'].count()
treatment_column

In [ ]:
# Look at 'work_interfere' value counts
work_inter_count = survey_df['work_interfere'].value_counts()
work_inter_count

In [ ]:
# total 'work_interfere'
work_inter_column = survey_df['work_interfere'].count()
work_inter_column

In [ ]:
# Replacing the NaN in the 'self_employed' and 'work_interfere' to a 'no' answer, assuming a non-answer means "No".
survey_df = survey_df.fillna({'self_employed': 'No',
                            'work_interfere': 'Never'})

survey_df.info()

In [ ]:
# recheck 'self_employed' value counts
self_emp_count = survey_df['self_employed'].value_counts()
self_emp_count

In [ ]:
# recheck 'work_interfere' value counts
work_inter_count = survey_df['work_interfere'].value_counts()
work_inter_count

In [ ]:
# Look at 'no_employess' value counts
employees_count = survey_df['no_employees'].value_counts()
employees_count

In [ ]:
# total 'no_employess'
employees_column = survey_df['no_employees'].count()
employees_column

In [ ]:
# Look at 'remote_work' value counts
remote_work_count = survey_df['remote_work'].value_counts()
remote_work_count

In [ ]:
# total 'remote_work'
remote_work_column = survey_df['remote_work'].count()
remote_work_column

In [ ]:
# Look at 'tech_company' value counts
tech_co_count = survey_df['tech_company'].value_counts()
tech_co_count

In [ ]:
# total 'tech_company'
tech_co_column = survey_df['tech_company'].count()
tech_co_column

In [ ]:
# Look at 'benefits' value counts
benefits_count = survey_df['benefits'].value_counts()
benefits_count

In [ ]:
# total 'benefits'
benefits_column = survey_df['benefits'].count()
benefits_column

In [ ]:
# Look at 'care_options' value counts
care_opt_count = survey_df['care_options'].value_counts()
care_opt_count

In [ ]:
# total 'care_options'
care_opt_column = survey_df['care_options'].count()
care_opt_column

In [ ]:
# Look at 'wellness_program' value counts
well_prog_count = survey_df['wellness_program'].value_counts()
well_prog_count

In [ ]:
# total 'wellness_program'
well_prog_column = survey_df['wellness_program'].count()
well_prog_column

In [ ]:
# Look at 'seek_help' value counts
help_count = survey_df['seek_help'].value_counts()
help_count

In [ ]:
# total 'seek_help'
help_column = survey_df['seek_help'].count()
help_column

In [ ]:
# Look at 'anonymity' value counts
anon_count = survey_df['anonymity'].value_counts()
anon_count

In [ ]:
# total 'anonymity'
anon_column = survey_df['anonymity'].count()
anon_column

In [ ]:
# Look at 'leave' value counts
leave_count = survey_df['leave'].value_counts()
leave_count

In [ ]:
# total 'leave'
leave_column = survey_df['leave'].count()
leave_column

In [ ]:
# Look at 'mental_health_consequence' value counts
mental_health_conseq = survey_df['mental_health_consequence'].value_counts()
mental_health_conseq

In [ ]:
# total 'mental_health_consequence'
mental_health_conseq_tl = survey_df['mental_health_consequence'].count()
mental_health_conseq_tl

In [ ]:
# Look at 'phys_health_consequence' value counts
phys_health_conseq = survey_df['phys_health_consequence'].value_counts()
phys_health_conseq

In [ ]:
# total 'phys_health_consequence'
phys_health_conseq_tl = survey_df['phys_health_consequence'].count()
phys_health_conseq_tl

In [ ]:
# Look at 'coworkers' value counts
coworker_trust = survey_df['coworkers'].value_counts()
coworker_trust

In [ ]:
# total 'coworkers'
coworker_trust_tl = survey_df['coworkers'].count()
coworker_trust_tl

In [ ]:
# Look at 'supervisor' value counts
supervisor_trust = survey_df['supervisor'].value_counts()
supervisor_trust

In [ ]:
# total 'supervisor'
supervisor_trust_tl = survey_df['supervisor'].count()
supervisor_trust_tl

In [ ]:
# Look at 'mental_health_interview' value counts
mental_health_int = survey_df['mental_health_interview'].value_counts()
mental_health_int

In [ ]:
# total 'mental_health_interview'
mental_health_int_tl = survey_df['mental_health_interview'].count()
mental_health_int_tl

In [ ]:
# Look at 'phys_health_interview' value counts
phys_health_int = survey_df['phys_health_interview'].value_counts()
phys_health_int

In [ ]:
# total 'phys_health_interview'
phys_health_int_tl = survey_df['phys_health_interview'].count()
phys_health_int_tl

In [ ]:
# Look at 'mental_vs_physical' value counts
mental_vs_phys = survey_df['mental_vs_physical'].value_counts()
mental_vs_phys

In [ ]:
# total 'mental_vs_physical'
mental_vs_phys_tl = survey_df['mental_vs_physical'].count()
mental_vs_phys_tl

In [ ]:
# Look at 'obs_consequence' value counts
obs_conseq = survey_df['obs_consequence'].value_counts()
obs_conseq

In [ ]:
# total 'obs_consequence'
obs_conseq_tl = survey_df['obs_consequence'].count()
obs_conseq_tl

In [ ]:
# Preview updated DataFrame
survey_df.head()